<a href="https://colab.research.google.com/github/maraperera/Easy-Lessons-Online/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   !pip install tf-nightly
# except Exception:
#   pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load the datasets
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'text'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'text'])

# Map 'ham' to 0 and 'spam' to 1
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Extract messages and labels
train_labels = train_df['label'].values
train_messages = train_df['text'].values

test_labels = test_df['label'].values
test_messages = test_df['text'].values

# Tokenization and Padding
vocab_size = 10000 # Max number of words to use
embedding_dim = 32 # Increased embedding dimension
max_len = 128 # Maximum sequence length

tokenizer = keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token="<unk>")
tokenizer.fit_on_texts(train_messages)

train_sequences = tokenizer.texts_to_sequences(train_messages)
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')

test_sequences = tokenizer.texts_to_sequences(test_messages)
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

# Build the improved model
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    keras.layers.Bidirectional(keras.layers.LSTM(32)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model with more epochs
num_epochs = 20 # Increased number of epochs
print(f"\nTraining model for {num_epochs} epochs...")
history = model.fit(train_padded, train_labels,
                    epochs=num_epochs,
                    validation_data=(test_padded, test_labels),
                    verbose=1)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction score and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # 1. Convert text to sequence of integers using the fitted tokenizer
  sequence = tokenizer.texts_to_sequences([pred_text])

  # 2. Pad the sequence to match the input length (max_len) expected by the model.
  padded_sequence = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len, padding='post', truncating='post')

  # Get prediction score from the model.
  prediction_score = model.predict(padded_sequence)[0][0]

  # Determine label based on the prediction score
  if prediction_score >= 0.5:
    label = 'spam'
  else:
    label = 'ham'

  # Return the prediction score
  return [float(prediction_score), label]

pred_text_example = "how are you doing today?"
prediction_example = predict_message(pred_text_example)
print(f"\nPrediction for '{pred_text_example}': {prediction_example}")

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
